In [9]:
from pyspark.sql import SparkSession
from operator import add
import pandas as pd
import string
import re

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.250:7077") \
        .appName("Nikitha_Krishna Murthy_lab 3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 17:17:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Question A.1

### A.1.1 Read the English transcripts with Spark, and count the number of lines

In [10]:
#reading the english transcript lines for finnish language and then counting number of lines
RDD_EN = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.fi-en.en")
print(RDD_EN.count())

1924942


### A.1.2 Do the same with the other language

In [11]:
#reading the FINNISH transcript lines for finnish language and then counting number of lines
RDD_FI = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.fi-en.fi")
print(RDD_FI.count())

1924942


### A.1.3 Verify that the line counts are the same for the two languages

In [12]:
if RDD_EN.count() == RDD_FI.count(): #checking condition for line count 
    print("the lines counts are same for english transcript and finnish transcript:",RDD_EN.count())
else:
    print("line count does not match")

the lines counts are same for english transcript and finnish transcript: 1924942


### A.1.4 Count the number of partitions.

In [13]:
RDD_ENpart=RDD_EN.getNumPartitions()#number of partitions for first RDD of english transcript
RDD_FIpart=RDD_FI.getNumPartitions()#number of partitions for first RDD of finnish transcript
print("Number of partitions for english transcript RDD:",RDD_ENpart)
print("Number of partitions for english transcript RDD:",RDD_FIpart)

Number of partitions for english transcript RDD: 3
Number of partitions for english transcript RDD: 3


### Question A.2

### A.2.1 Pre-process the text from both RDDs by doing the following:
###  ● Lowercase the text
###  ● Tokenize the text (split on space)
###  Hint: define a function to run in your driver application to avoid writing this code twice

In [14]:
def tokenizer_lower(lines):
    #X=lines.translate(str.maketrans('','',string.punctuation))
    X=re.sub(r'[^\w]'," ",lines)
    return X.lower().split()

### A.2.2 Inspect 10 entries from each of your RDDs to verify your pre-processing.

In [15]:
#printing first 10 lines preprocessed English transcript in lowercase and after split
preprocess_EN=RDD_EN.map(tokenizer_lower)
print("the first 10 lines for preprocessed english transcript in lowercase are:")
for line in preprocess_EN.take(10):
    print(line)

the first 10 lines for preprocessed english transcript in lowercase are:
['resumption', 'of', 'the', 'session']
['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period']
['although', 'as', 'you', 'will', 'have', 'seen', 'the', 'dreaded', 'millennium', 'bug', 'failed', 'to', 'materialise', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful']
['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days', 'during', 'this', 'part', 'session']
['in', 'the', 'meantime', 'i', 'should', 'like', 'to', 'observe', 'a', 'minute', 's', 'silence', 'as', 'a', 'number', 'of'

In [16]:
#printing first 10 lines preprocessed finnish transcript in lowercase and after split
preprocess_FI=RDD_FI.map(tokenizer_lower)
print("the first 10 lines for preprocessed finnish transcript in lowercase are:")
for line in preprocess_FI.take(10):
    print(line)

the first 10 lines for preprocessed finnish transcript in lowercase are:


['istuntokauden', 'uudelleenavaaminen']
['julistan', 'perjantaina', 'joulukuun', '17', 'päivänä', 'keskeytetyn', 'euroopan', 'parlamentin', 'istunnon', 'avatuksi', 'ja', 'esitän', 'vielä', 'kerran', 'vilpittömän', 'toiveeni', 'siitä', 'että', 'teillä', 'olisi', 'ollut', 'oikein', 'mukava', 'joululoma']
['kuten', 'olette', 'varmaan', 'saattaneet', 'huomata', 'vuodenvaihteeseen', '2000', 'povattuja', 'suuria', 'tietokoneongelmia', 'ei', 'ilmennytkään', 'sen', 'sijaan', 'todella', 'kauheat', 'luonnonkatastrofit', 'koettelivat', 'kansalaisia', 'joissakin', 'unionimme', 'maissa']
['te', 'olette', 'esittäneet', 'toiveen', 'että', 'tästä', 'asiasta', 'keskusteltaisiin', 'lähipäivinä', 'tämän', 'istuntojakson', 'aikana']
['sillä', 'välin', 'toivoisin', 'kuten', 'useampi', 'kollega', 'on', 'minulle', 'esittänytkin', 'että', 'viettäisimme', 'minuutin', 'hiljaisuuden', 'kaikkien', 'niiden', 'uhrien', 'muistoksi', 'jotka', 'saivat', 'surmansa', 'useita', 'euroopan', 'unionin', 'maita', 'koetelleis

### A.2.3 Verify that the line counts still match after the pre-processing.

In [17]:
X=preprocess_EN.count() #counting number lines in EN preprocessed transcripts.
Y=preprocess_FI.count() #counting number lines in FN preprocessed transcripts.
print("line counts of preprocessed english transcript:",X)
print("line counts of preprocessed finnish transcript:",Y)
if X==Y:
    print("line count matches in english and finnish transcript")
else:
    print("line count does not match")


line counts of preprocessed english transcript: 1924942
line counts of preprocessed finnish transcript: 1924942
line count matches in english and finnish transcript


### Question 1.A.3

### A.3.1 Use Spark to compute the 10 most frequently according words in the English language corpus. Repeat for the other language. 

In [18]:
#function to count 10 frequently occuring words
def word_occurance(X):
    flat_X = X.flatMap(lambda i: i) #using flatmap to convert the words into single list
    map_X = flat_X.map(lambda i: (i,1))#to map words to number 1
    reduce_X = map_X.reduceByKey(add)#to count the words
    return reduce_X.takeOrdered(10,key=lambda i: -i[1]) #to print the words at top 10 occurences

In [38]:
##printing top 10 most frequently occured words in english transcript
Count_EN=word_occurance(preprocess_EN)
print("he 10 most frequently occured words in english transcript are :")
X=pd.DataFrame(Count_EN,columns=["word","count"])
print(X)

#printing top 10 most frequently occured words in Danish transcript
Count_FI=word_occurance(preprocess_FI)
print("The 10 most frequently occured words in finnish transcript are :")
Y=pd.DataFrame(Count_FI,columns=["word","count"])
print(Y)

he 10 most frequently occured words in english transcript are :
   word    count
0   the  3639601
1    of  1727482
2    to  1605863
3   and  1371958
4    in  1141428
5  that   873391
6     a   807440
7    is   803006
8   for   558170
9    we   552003


The 10 most frequently occured words in finnish transcript are :
       word    count
0        ja  1252346
1        on  1052444
2      että   620097
3  euroopan   263163
4        ei   252346
5      myös   185597
6        se   183997
7      ovat   163770
8   arvoisa   149835
9       sen   145099


### A.3.2 Verify that your results are reasonable.

Downloaded the dataset for english and finnish transcript from link : https://www.statmt.org/europarl/v6/fi-en.tgz into Instance VM using wget command. Then calculated the word count of 10 most frequently occured words in english and finnish transcript using commands : 
### sed -e 's/[^[:alpha:]]/ /g' europarl-v6.fi-en.en | tr '\n' " " |  tr -s " " | tr " " '\n'| tr 'A-Z' 'a-z' | sort | uniq -c | sort -nr | nl | head -n 10
### sed -e 's/[^[:alpha:]]/ /g' europarl-v6.fi-en.fi | tr '\n' " " |  tr -s " " | tr " " '\n'| tr 'A-Z' 'a-z' | sort | uniq -c | sort -nr | nl | head -n 10
The result obtained from above commands provides the same top 10 word accurances as above with approximately similar count.
Hence the result abtained seems to resonable.



### Question A.4

### A.4.1  Use this parallel corpus to mine some translations in the form of word pairs, for the two languages. Do this by pairing words found on short lines with the same number of words respectively. We (incorrectly) assume the words stay in the same order when translated.


### Follow this approach. Work with the pair of RDDs you created in question A.2. Hint: make a new pair of RDDs for each step, sv_1, en_1, sv_2, en_2, ...

### 1. Key the lines by their line number (hint: ZipWithIndex()).

In [100]:
EN_1=preprocess_EN.zipWithIndex()
print(EN_1.take(10))
FI_1=preprocess_FI.zipWithIndex()
print(FI_1.take(10))



[(['resumption', 'of', 'the', 'session'], 0), (['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period'], 1), (['although', 'as', 'you', 'will', 'have', 'seen', 'the', 'dreaded', 'millennium', 'bug', 'failed', 'to', 'materialise', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful'], 2), (['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days', 'during', 'this', 'part', 'session'], 3), (['in', 'the', 'meantime', 'i', 'should', 'like', 'to', 'observe', 'a', 'minute', 's', 'silence', 'as', 'a', 'number', 'of', 'members', 'have', 'requested', 'on', 'behalf

[(['istuntokauden', 'uudelleenavaaminen'], 0), (['julistan', 'perjantaina', 'joulukuun', '17', 'päivänä', 'keskeytetyn', 'euroopan', 'parlamentin', 'istunnon', 'avatuksi', 'ja', 'esitän', 'vielä', 'kerran', 'vilpittömän', 'toiveeni', 'siitä', 'että', 'teillä', 'olisi', 'ollut', 'oikein', 'mukava', 'joululoma'], 1), (['kuten', 'olette', 'varmaan', 'saattaneet', 'huomata', 'vuodenvaihteeseen', '2000', 'povattuja', 'suuria', 'tietokoneongelmia', 'ei', 'ilmennytkään', 'sen', 'sijaan', 'todella', 'kauheat', 'luonnonkatastrofit', 'koettelivat', 'kansalaisia', 'joissakin', 'unionimme', 'maissa'], 2), (['te', 'olette', 'esittäneet', 'toiveen', 'että', 'tästä', 'asiasta', 'keskusteltaisiin', 'lähipäivinä', 'tämän', 'istuntojakson', 'aikana'], 3), (['sillä', 'välin', 'toivoisin', 'kuten', 'useampi', 'kollega', 'on', 'minulle', 'esittänytkin', 'että', 'viettäisimme', 'minuutin', 'hiljaisuuden', 'kaikkien', 'niiden', 'uhrien', 'muistoksi', 'jotka', 'saivat', 'surmansa', 'useita', 'euroopan', 'unio

### 2. Swap the key and value - so that the line number is the key.

In [101]:
EN_2=EN_1.map(lambda i: (i[1],i[0]))
FI_2=FI_1.map(lambda i: (i[1],i[0]))
print(EN_2.take(10),FI_2.take(10))

[(0, ['resumption', 'of', 'the', 'session']), (1, ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period']), (2, ['although', 'as', 'you', 'will', 'have', 'seen', 'the', 'dreaded', 'millennium', 'bug', 'failed', 'to', 'materialise', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful']), (3, ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days', 'during', 'this', 'part', 'session']), (4, ['in', 'the', 'meantime', 'i', 'should', 'like', 'to', 'observe', 'a', 'minute', 's', 'silence', 'as', 'a', 'number', 'of', 'members', 'have', 'requested', 'on', 'beh

### 3. Join the two RDDs together according to the line number key, so you have pairs of matching lines.

In [102]:
ENFI_transcript=EN_2.join(FI_2)
print(ENFI_transcript.take(10))

[(20922, (['finally', 'mr', 'president', 'we', 'have', 'to', 'guarantee', 'a', 'combination', 'of', 'policies', 'which', 'promote', 'growth', 'and', 'employment', 'guarantee', 'macroeconomic', 'security', 'and', 'the', 'consolidation', 'of', 'the', 'euro', 'which', 'promote', 'growth', 'and', 'the', 'transition', 'towards', 'a', 'society', 'and', 'economy', 'of', 'innovation', 'and', 'knowledge'], ['arvoisa', 'puhemies', 'päätän', 'sanomalla', 'että', 'meidän', 'on', 'taattava', 'sellaisten', 'politiikkojen', 'yhdistelmä', 'jotka', 'suosivat', 'kasvua', 'ja', 'työllisyyttä', 'varmistavat', 'makrotaloudellisen', 'vakauden', 'ja', 'vahvistavat', 'euroa', 'jotka', 'edistävät', 'kasvua', 'ja', 'siirtymistä', 'innovaatioiden', 'sekä', 'tietojen', 'ja', 'taitojen', 'yhteiskuntaan', 'ja', 'talouteen'])), (21528, (['the', 'damage', 'caused', 'by', 'lorries', 'in', 'traffic', 'accidents', 'is', 'more', 'often', 'than', 'not', 'considerable', 'and', 'the', 'state', 'of', 'vehicles', 'is', 'very'

### 4. Filter to exclude line pairs that have an empty/missing “corresponding” sentence

In [103]:
ENFI_filter1=ENFI_transcript.filter(lambda i: i[1][0]!=[] and i[1][1]!=[])
print(ENFI_filter1.take(10))


[(20922, (['finally', 'mr', 'president', 'we', 'have', 'to', 'guarantee', 'a', 'combination', 'of', 'policies', 'which', 'promote', 'growth', 'and', 'employment', 'guarantee', 'macroeconomic', 'security', 'and', 'the', 'consolidation', 'of', 'the', 'euro', 'which', 'promote', 'growth', 'and', 'the', 'transition', 'towards', 'a', 'society', 'and', 'economy', 'of', 'innovation', 'and', 'knowledge'], ['arvoisa', 'puhemies', 'päätän', 'sanomalla', 'että', 'meidän', 'on', 'taattava', 'sellaisten', 'politiikkojen', 'yhdistelmä', 'jotka', 'suosivat', 'kasvua', 'ja', 'työllisyyttä', 'varmistavat', 'makrotaloudellisen', 'vakauden', 'ja', 'vahvistavat', 'euroa', 'jotka', 'edistävät', 'kasvua', 'ja', 'siirtymistä', 'innovaatioiden', 'sekä', 'tietojen', 'ja', 'taitojen', 'yhteiskuntaan', 'ja', 'talouteen'])), (21528, (['the', 'damage', 'caused', 'by', 'lorries', 'in', 'traffic', 'accidents', 'is', 'more', 'often', 'than', 'not', 'considerable', 'and', 'the', 'state', 'of', 'vehicles', 'is', 'very'

### 5. Filter to leave only pairs of sentences with a small number of words per sentence, this should give a more reliable translation.

In [104]:
ENFI_filter2=ENFI_filter1.filter(lambda i: len(i[1][0])<=9 and len(i[1][1])<=9)
print(ENFI_filter2.take(10))

[(927948, (['a', 'new', 'approach', 'is', 'needed'], ['vaadimme', 'uutta', 'lähtökohtaa'])), (990144, (['many', 'speakers', 'expressed', 'their', 'wish', 'for', 'a', 'social', 'europe'], ['useat', 'puhujat', 'ilmaisivat', 'toivovansa', 'sosiaalista', 'eurooppaa'])), (1075812, (['however', 'that', 'is', 'not', 'all'], ['tämä', 'ei', 'ole', 'vielä', 'kaikki', 'pakotteet', 'ovat', 'väline'])), (1081746, (['this', 'is', 'an', 'area', 'where', 'more', 'work', 'is', 'needed'], ['tällä', 'alalla', 'on', 'tehtävä', 'enemmän', 'työtä'])), (1092276, (['that', 'should', 'be', 'made', 'clear', 'outside', 'the', 'eu'], ['tämä', 'pitäisi', 'tehdä', 'selväksi', 'eu', 'n', 'ulkopuolella'])), (1147932, (['international', 'attention', 'tends', 'to', 'focus', 'on', 'other', 'things'], ['kansainvälinen', 'huomio', 'tapaa', 'kiinnittyä', 'muihin', 'asioihin'])), (1220658, (['applause'], ['suosionosoituksia'])), (1230582, (['i', 'have', 'just', 'one', 'other', 'query'], ['minulla', 'on', 'vielä', 'yksi', 't

### 6. Filter to leave only pairs of sentences with the same number of words in each sentence.

In [105]:
ENFI_filter3=ENFI_filter2.filter(lambda i: len(i[1][0]) == len(i[1][1]))
print(ENFI_filter3.take(10))


[(987096, (['written', 'statements', 'rule', '142'], ['kirjalliset', 'kannanotot', '142', 'artikla'])), (1084026, (['welcome'], ['tervetulotoivotukset'])), (1264920, (['these', 'are', 'very', 'complicated', 'scientific', 'issues'], ['nämä', 'ovat', 'hyvin', 'monimutkaisia', 'tieteellisiä', 'kysymyksiä'])), (1272018, (['this', 'situation', 'must', 'change'], ['tämän', 'tilanteen', 'on', 'muututtava'])), (1355208, (['imports', 'are', 'necessary'], ['tuonti', 'on', 'välttämätöntä'])), (1639674, (['i', 'think', 'we', 'can', 'be', 'reasonably', 'optimistic'], ['lisäksi', 'arvoisa', 'puhemies', 'mielestäni', 'pitää', 'olla', 'optimistinen'])), (1130190, (['however', 'it', 'must', 'be', 'economically', 'sustainable'], ['sen', 'täytyy', 'kuitenkin', 'olla', 'taloudellisesti', 'kestävää'])), (902526, (['6'], ['6'])), (910848, (['the', 'status', 'quo', 'is', 'not', 'sustainable'], ['status', 'quo', 'ei', 'ole', 'kestävä', 'ratkaisu'])), (953808, (['14'], ['14']))]


### 7. For each sentence pair, map so that you pair each (in order) word in the two sentences. We no longer need the line numbers. (hint: use python’s built in zip()function)

In [106]:
ENFI_filter4 = ENFI_filter3.map(lambda i: list(zip(i[1][0],i[1][1])))
print(ENFI_filter4.take(10))

[[('report', 'klichin'), ('klich', 'mietintö')], [('why', 'miksi'), ('not', 'ei')], [('however', 'tämä'), ('this', 'vuosi'), ('year', 'ei'), ('is', 'kuitenkaan'), ('no', 'ole'), ('ordinary', 'tavallinen'), ('year', 'vuosi')], [('applause', 'suosionosoituksia')], [('the', 'kysymys'), ('question', 'palestiinalaisten'), ('of', 'keskinäisen'), ('palestinian', 'sovinnon'), ('reconciliation', 'löytymisestä'), ('will', 'on'), ('also', 'myös'), ('be', 'keskeisellä'), ('central', 'sijalla')], [('1', '1')], [('right', 'metsät'), ('now', 'tarvitsevat'), ('forests', 'juuri'), ('need', 'nyt'), ('even', 'entistä'), ('more', 'enemmän'), ('protection', 'suojelua')], [('7', '7')], [('7', '7')], [('noise', 'hälinää')]]


### 8. Use reduce to count the number of occurrences of the word-translation-pairs.

In [113]:
red_occurance = word_occurance(ENFI_filter4)#calling reduce function to count number of occurences.

the most frequently occured words in English and Finnish transcript are :
                            word  count
0                       (is, on)   4237
1  (applause, suosionosoituksia)   3626
2                      (and, ja)   1464
3                   (this, tämä)   1235
4             (report, mietintö)   1221
5           (debate, keskustelu)   1001
6                       (is, ei)    883
7            (closed, päättynyt)    881
8                       (it, se)    847
9                     (not, ole)    784


### 9.Print some of the most frequently occurring pairs of words.

In [112]:
print("the most frequently occured words in English and Finnish transcript are :")
X=pd.DataFrame(red_occurance,columns=["word","count"])
print(X)

the most frequently occured words in English and Finnish transcript are :
                            word  count
0                       (is, on)   4237
1  (applause, suosionosoituksia)   3626
2                      (and, ja)   1464
3                   (this, tämä)   1235
4             (report, mietintö)   1221
5           (debate, keskustelu)   1001
6                       (is, ei)    883
7            (closed, päättynyt)    881
8                       (it, se)    847
9                     (not, ole)    784


### Do your translations seem reasonable? Use a dictionary to check a few (don’t worry, you won’t be marked down for incorrect translations).

The translation seems to be happening correctly for about 70%, checked with google translator. The word päättynyt means finish but it is showing as closed(which is almost the same).The other translations seems to perfect and reasonable.

In [115]:
spark_context.stop()